In [ ]:
import polars as pl
from function.clean_spm import clean
from function.preprocessing import spm
from function.descriptive import descriptive
from function.visualisation import visual
import plotly.express as px
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import skew, kurtosis, shapiro, norm, spearmanr
import numpy as np

# read the spm_database
df = clean.read_spm_database(initial_clean=True, parquet_file = True)

# To generate a full list of STR beneficiaries
long_df = pl.concat([df.select("id", "id_ben", "sex_beneficiary", "ori_ben_age", "str_category", "state", "code_parlimen")\
                       .rename({"ori_ben_age":"age", "sex_beneficiary":"sex"})\
                       .with_columns(pl.col("age").cast(pl.Int64)),
                     df.filter(pl.col("id_partner").is_not_null())\
                       .select("id", "id_partner", "sex_partner", "age_partner", "str_category", "state", "code_parlimen")\
                          .rename({"id_partner":"id_ben", "sex_partner":"sex", "age_partner":"age"})\
                            .with_columns(pl.col("age").cast(pl.Int64)),
                     df.filter(pl.col("id_dependent").is_not_null())\
                       .select("id", "id_dependent", "sex_dependent", "age_dependent", "str_category", "state", "code_parlimen")\
                          .rename({"id_dependent":"id_ben", "sex_dependent":"sex", "age_dependent":"age"})\
                            .with_columns(pl.col("age").cast(pl.Int64))])\
          .unique(subset='id_ben')

In [ ]:
old_df = long_df.filter(pl.col("age") < 2000, pl.col("age") > 100, pl.col("str_category") == "Household")
old_df

In [ ]:
# df.filter(pl.col("age_dependent") > 2000)
ori_df = clean.read_spm_database(initial_clean=True, parquet_file = False)

In [ ]:
# To show there is some without address, which will be drop out
long_df.null_count()

In [ ]:
# Count the partner
print(df.filter(pl.col("id_partner").is_not_null())\
.select("id_ben", "id_partner").unique().count())

print(df.filter(pl.col("id_partner").is_not_null())\
.select("id_ben", "id_partner").unique(subset = "id_ben").count())

print(df.filter(pl.col("id_partner").is_not_null())\
.select("id_ben", "id_partner").unique(subset = "id_partner").count())

# Count the dependent
print(df.filter(pl.col("id_dependent").is_not_null())\
.select("id_ben", "id_dependent").unique().count())

print(df.filter(pl.col("id_dependent").is_not_null())\
.select("id_ben", "id_dependent").unique(subset = "id_ben").count())

print(df.filter(pl.col("id_dependent").is_not_null())\
.select("id_ben", "id_dependent").unique(subset = "id_dependent").count())

In [ ]:
# To show distribution of sex
sex_pt = descriptive.gender_distribution(df)
sex_pt

In [ ]:
# Distribution of age
age_df = long_df.filter(pl.col("age").is_not_null(), 
                        pl.col("age") < 100).select("age", "sex").to_pandas()

# Using pandas default describe
descriptive_df = age_df.describe().apply(lambda s: s.apply('{0:,.2f}'.format))

# Calculate the skew and kurtosis
for formula in [np.var, skew, kurtosis]:
    descriptive_df.loc[f"{formula.__name__}"] = [formula(age_df.loc[:,column]) for column in descriptive_df.columns]

# Print the table
print(descriptive_df.apply(lambda s: s.apply('{0:,.2f}'.format)).to_markdown(tablefmt="pretty"))

# To plot the graph
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(15, 3))

# Histogram
ax1.hist(age_df.loc[:,"age"], bins=len(age_df.loc[:,"age"].unique()), edgecolor='black')
ax1.set_title(f'Histogram for {"age"}')
ax1.set_xlabel("age")
ax1.set_ylabel('Frequency')

# QQ plot
sm.qqplot(age_df.loc[:,"age"], ax=ax2, line='45', fit = True)
ax2.set_title(f'QQ Plot for {"age"}')

# Box plot
sns.boxplot(age_df.loc[:,"age"], ax=ax3)
ax3.set_title(f'Box Plot for {"age"}')

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.3)

# Display the plot
plt.show()

In [ ]:
px.histogram(age_df, x="age")
px.box(age_df.loc[:,"age"])

In [ ]:
# To show distribution of state
state_pt = long_df.group_by("state").len("count")\
                .with_columns((pl.col("count")/len(long_df) * 100).round(2).alias("count_%"))

# Print the result in table format
print(state_pt)

# Plot the chorepleth
visual.draw_chorepleth(map_file_type="state",
                       df = long_df,
                       location = "state",
                       z="len",
                       featureidkey="state",
                       text=("state",),
                       colorbar_title="STR Population Density")

In [ ]:
# To show distribution of code_parlimen
parlimen_pt = long_df.group_by("code_parlimen").len("count")\
                .with_columns((pl.col("count")/len(long_df) * 100).round(2).alias("count_%"))

# Print the result in table format
# print(parlimen_pt)

# Plot the chorepleth
visual.draw_chorepleth(map_file_type="parlimen",
                       df = long_df,
                       location = "code_parlimen",
                       z="len",
                       featureidkey="code_parlimen",
                       text=("parlimen","state"),
                       colorbar_title="STR Population Density")

In [ ]:
from function.visualisation import visual
import polars as pl

gp_df = pl.read_excel(visual._gp_file)

visual.draw_chorepleth(map_file_type="district", df=gp_df, location="code_state_district", z="len", featureidkey="code_state_district")

In [ ]:
visual.gp_scatter(query=None, text=None, color=("district"), mapbox_style="basic", height=600)

In [ ]:
pl.read_excel(visual._gp_file).group_by("district").len("Number of GPs").to_pandas().sort_values("Number of GPs", ascending=False).to_clipboard(index=False)

In [ ]:
px.density_mapbox(population, lat="Latitude", lon="Longitude", z="pov_headcount", mapbox_style="open-street-map", radius = 20, zoom=5, height=600)\
.add_trace(px.scatter_mapbox(clinic_data, lat="Latitude", lon="Longitude").data[0])

In [ ]:
 # Import necessary packages
import json
import plotly.graph_objects as go

# use json to load the choropleth file
with open(visual.read_map_file(map_file_type="population", read_with_gpd=False)) as file:
    geojson_data = json.load(file)
# geojson_data["features"][0]

px.choropleth(population, geojson=geojson_data, locations="fid", color="pop2023p_adj_2km", 
              color_continuous_scale="Viridis", range_color=(0,12))
population

In [ ]:
import polars as pl

pd.read_parquet("data/population/Household Income Districts.parquet")\
    .merge(visual.read_map_file("district", read_with_gpd=True), on="district")

In [ ]:
import plotly.express as px
import pandas as pd
from function.visualisation import visual
from dotenv import load_dotenv
import os

# To set the environement
load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

px.density_mapbox(pd.read_csv("data/map_file/mys_pd_2020_1km_ASCII_XYZ.csv"),
                  lat="Y", lon="X", z="Z", radius=5,
                  labels = {"Z":"Population density per 1km"},
                  mapbox_style="open-street-map", height = 600,
                  zoom=5)\
.add_trace(px.scatter_mapbox(clinic_data, lat="Latitude", lon="Longitude", color="district", mapbox_style="open-street-map").data[0])

In [ ]:
import pandas as pd
import geopandas as gpd
import rasterio
from shapely.geometry import Point
import matplotlib.pyplot as plt
from function.visualisation import visual
import plotly.express as px

# , 
district_list = ['14_1', '10_8', '10_1', '10_5', '10_2', '8_3', '7_4', '1_2']

clinic_data = pd.read_excel(visual._gp_file)
clinic_data['geometry'] = clinic_data.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
clinics = gpd.GeoDataFrame(clinic_data, geometry='geometry', crs='EPSG:4326')  # Assuming WGS84

# with rasterio.open("data/map_file/Malaysia Population Density 2020.tif") as src:
#     population_density = src.read(1)
#     transform = src.transform
#     crs = src.crs

population = visual.read_map_file("population", read_with_gpd=True).set_geometry("geometry")
# population["Latitude"] = population["geometry"].centroid.y
# population["Longitude"] = population["geometry"].centroid.x

def calculate_average_nearest_neighbor(clinic_data, population_data):
    # Convert clinic data to GeoPandas GeoDataFrame
    # clinic_gdf = gpd.GeoDataFrame(clinic_data, 
    #                               geometry=gpd.points_from_xy(clinic_data["Longitude"], clinic_data["Latitude"]))
    
    # Ensure both GeoDataFrames have the same CRS
    # clinic_gdf = clinic_gdf.to_crs('EPSG:4326')

    # Calculate nearest neighbor distances
    distances = []
    for index, clinic in clinic_data.iterrows():
        nearest_distance = clinic["geometry"].distance(population_data.geometry)
        distances.append(nearest_distance)

    # Calculate Average Nearest Neighbor Distance
    return sum(distances) / len(distances)

temp_df = pd.DataFrame()
for district in district_list:
    print(district)
    temp_population = population.query(f"code_state_district == '{district}'")
    ann_temp = calculate_average_nearest_neighbor(clinic_data=clinics.query(f"code_state_district == '{district}'"), 
                                                  population_data=temp_population)

    temp_population.loc[:,"ann"] = ann_temp

    temp_df = pd.concat([temp_df, temp_population.loc[:,("fid", "code_state_district", "ann")]])    

temp_df

In [ ]:
import json
with open(visual.read_map_file(map_file_type="population", read_with_gpd=False)) as file:
    geojson_data = json.load(file)

kl_population = population.query("code_state_district == '14_1'")
kl_gp = clinic_data.query("state == 'W.P. Kuala Lumpur'")

kl_population["lat"] = kl_population["geometry"].centroid.y
kl_population["lon"] = kl_population["geometry"].centroid.x

# px.choropleth_mapbox(kl_population, geojson=geojson_data, featureidkey="fid", locations="fid", color="pov_headcount")
px.density_mapbox(kl_population, lat="lat", lon="lon", radius=20, z="pov_headcount", mapbox_style="open-street-map")

In [ ]:
district_df = visual.read_map_file("district", read_with_gpd=True)

gpd.sjoin(clinics, district_df)["district_right"].unique()

In [ ]:
# population.loc[:,"ann"] = ann_list
# px.histogram(population.query("code_state_district.isin(['1_2'])"), 
#              x="ann", nbins = 50)
population

In [ ]:
pl.read_parquet("""data/population/Malaysia Population Table.parquet""")\
.filter(pl.col("date").cast(pl.String) == '2023-01-01')\
.with_columns((pl.col("population")/33379.5 * 100).alias("percentage")).head(7)

In [ ]:
# No need to write for all?
# Concentrate on the population first, for 10 districts,
# Merge with parliment to get the parliment involved
# Divide the parliment str number by total population to get a percentage
# Use the percentage to times with the z value from ASCII for density

In [ ]:
from function.file import file
from function.provider import gp
from function.population import population
import geopandas as gpd
import pandas as pd
import polars as pl

parlimen_population = pl.read_parquet("data/population/Population Parlimen.parquet")
code_parlimen_district = pl.from_pandas(gpd.read_file(file._map_parlimen).sjoin(
                              gpd.read_file(file._map_district).drop(columns = ["state", "code_state"]))\
                                    .loc[:,("code_parlimen", "district", "code_state_district")])

temp_df = parlimen_population.filter(pl.col("date").cast(pl.String) == "2022-01-01",
                                     pl.col("sex") == "both",
                                     pl.col("age") == "overall",
                                     pl.col("ethnicity") == "overall")\
                             .with_columns(pl.col("population") * 1000).to_pandas()
code_parlimen_df = gpd.read_file(file._map_parlimen).loc[:,("parlimen", "code_parlimen")]
temp_df = pl.from_pandas(temp_df.merge(code_parlimen_df, how="left", on="parlimen"))

long_df = population.convert_str_to_long(df = pl.read_parquet(file._file_spm_parquet))

percentage_df = temp_df.join(long_df.group_by("code_parlimen").len("str_count"),
                             how="left", on="code_parlimen")\
                       .select("parlimen", "code_parlimen", (pl.col("str_count")/pl.col("population")).alias("str_percentage"))

parlimen_population.join(percentage_df, how="left", on="parlimen")\
.join(code_parlimen_district, how="left", on="code_parlimen")\
.write_parquet("data/population/parlimen_district_str.parquet", use_pyarrow=True)

In [ ]:
gpd.read_file(file._map_parlimen).sjoin(gpd.read_file(file._map_district).drop(columns = ["state", "code_state"]))\
.loc[:,("code_parlimen", "district", "code_state_district")]

In [ ]:
pl.read_parquet(file._population_parlimen)

In [ ]:
from function.file import file
from function.provider import gp
from function.population import population
from function.map import map
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px

# # To get percentage for population according to lat lon
# parlimen_population = pl.read_parquet(file._population_parlimen)
# parlimen_population_geo = gpd.GeoDataFrame(data=parlimen_population.to_pandas()\
#                                                  .merge(gpd.read_file(file._map_parlimen).drop(columns="state"), 
#                                                         how="left", on="parlimen"),
#                                            geometry="geometry", crs='EPSG:4326')
# # long_df = population.convert_str_to_long(df = pl.read_parquet(file._file_spm_parquet)).group_by("code_parlimen").len("str_count")
# long_df = pl.read_parquet(file._file_spm_parquet).unique(subset="id_ben").group_by("code_parlimen").len("str_count")

# # Prepare on ASCII file
# population_ascii = map.convert_pandas_geopandas(pd.read_csv(file._population_ascii), lat="Y", lon="X")
# population_ascii.loc[:,"ascii_population"] = population_ascii.loc[:,"Z"] * 32447100 /population_ascii.Z.sum() * 1.0287

# # Filter the parliment population date
# temp_df = pl.from_pandas(parlimen_population_geo.drop(columns="geometry"))\
#             .filter(pl.col("date").cast(pl.Date).cast(pl.String) == "2020-01-01",
#                                         pl.col("sex") == "both",
#                                         pl.col("age") == "overall",
#                                         pl.col("ethnicity") == "overall")\
#                             .with_columns(pl.col("population") * 1000)\
#             .group_by("code_parlimen").agg(pl.col("population").sum())

# percentage_df = temp_df.join(long_df, on="code_parlimen")\
#                         .with_columns((pl.col("str_count")/pl.col("population")).alias("str_percentage"))

# # Merge both ascii and parlimen_population
# df = pl.from_pandas(gpd.sjoin(parlimen_population_geo, population_ascii)\
#                     .loc[:,("date", "state", "sex", "age", "ethnicity", "population", "code_parlimen", "parlimen", "X", "Y", "ascii_population")])\
#        .with_columns(pl.col("date").cast(pl.Date))\
#        .join(percentage_df.select("code_parlimen", "str_percentage"), how="left", on ="code_parlimen")\
#        .with_columns((pl.col("ascii_population") * pl.col("str_percentage")).alias("str_ascii"))

# df = map.convert_pandas_geopandas(df.to_pandas(), lat="Y", lon="X")

# final_df = df.sjoin(gpd.read_file(file._map_district).drop(columns="state")).drop(columns = ["geometry", "index_right"])

# final_df = pl.from_pandas(final_df)\
#     .with_columns(pl.col("date").cast(pl.Date))

population = pl.read_parquet(file._population_str_ascii_households)
# geojson_data = map.read_geojson_file(file._map_district)
population = population.filter(pl.col("sex")=="both",
                  pl.col("age")=="overall",
                  pl.col("ethnicity")=="overall",
                  pl.col("date").cast(pl.String)=="2022-01-01")\
.select("X","Y","ascii_population","str_percentage","str_ascii").to_pandas()

In [ ]:
px.density_mapbox(population,
                  lat="Y",lon="X", z="str_ascii", radius=10,
                  mapbox_style="open-street-map",
                  color_continuous_scale="rainbow",
                  center={"lat": 4.389059008652357, "lon": 108.65244272591418},
                  zoom=5
)

In [ ]:
from function.file import file
from function.provider import gp
from function.population import population
from function.map import map
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from dotenv import load_dotenv
import os

# To set the environement
load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

geojson_data = map.read_geojson_file(file._map_district)

str_parlimen = pl.read_parquet(file._file_spm_parquet)\
                 .unique(subset="id_ben")\
                 .group_by("code_parlimen").len("str_count").to_pandas()

district = gpd.read_file(file._map_district)
parlimen = gpd.read_file(file._map_parlimen)
# parlime_district = parlimen.sjoin(district.drop(columns="state")).drop(columns="index_right")
population_ascii = map.convert_pandas_geopandas(pd.read_csv(file._population_ascii),lat="Y",lon="X")
temp_population = population_ascii.sjoin(parlimen, how="inner")\
                                  .drop(columns=["state", "index_right", "code_state"])\
                                  .sjoin(district, how="inner")\
                                  .drop(columns=["index_right", "geometry"])
temp_population = temp_population.merge(temp_population.pivot_table(index="code_parlimen", values="Z", aggfunc=sum)\
                      .reset_index().rename(columns={"Z":"parlimen_z"}),
                      how="left", on="code_parlimen")

str_parlimen = pl.read_parquet(file._file_spm_parquet)\
                 .unique(subset="id_ben")\
                 .group_by("code_parlimen").len("str_count").to_pandas()

temp_population = temp_population.merge(str_parlimen, how="left", on="code_parlimen")

temp_population.loc[:,"estimated_str"] = temp_population.loc[:,"str_count"] * temp_population.loc[:,"Z"] / temp_population.loc[:,"parlimen_z"]

In [ ]:
def match_ppvgp(add_1):
    address = ppvgp_fac_code.apply(lambda x: fuzz.token_sort_ratio(add_1, x["ALAMAT"]), axis = 1)
    return ppvgp_fac_code.loc[address.idxmax(), "FACILITY CODE"]

# # Solve Facility Code via nearest geometry code
# def dist(lat1, long1, lat2, long2):
#     # Replicating the same formula as mentioned in Wiki convert decimal degrees to radians 
#     lat1, long1, lat2, long2 = map(radians, [lat1, long1, lat2, long2])
#     # Haversine formula 
#     dlon = long2 - long1
#     dlat = lat2 - lat1 
#     a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
#     c = 2 * asin(sqrt(a)) 
#     # Radius of earth in kilometers is 6371km
#     km = 6371* c
#     return km

# def find_nearest(lat, long):
#     distances = gp_df.apply(lambda row: dist(lat, long, row["Latitude"], row["Longitude"]), axis=1)
#     return gp_df.loc[distances.idxmin(), "id"]


# temp_population["id"] = temp_population.apply(lambda row: find_nearest(row["lat"], row["lon"]), axis = 1)
# temp_population

In [ ]:
from function.file import file
from function.provider import gp
from function.population import population
from function.map import map
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from dotenv import load_dotenv
import os

# To set the environement
load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

temp_population = pd.read_parquet(file._population_str_ascii_households)
temp_population.to_hdf("data/population/str_ascii_household.h5", index=False, key="str")

In [ ]:
fig = go.Figure(go.Densitymapbox(lat=temp_population.Y, lon=temp_population.X, z=temp_population.loc[:,"estimated_str"],
                                 radius=20,
                                 autocolorscale = False, 
                                 colorscale="rainbow",
                                 colorbar_title="STR Population",
                                 opacity=0.5,
                                 showlegend=False,
                                 text=None,
                                 zmin=temp_population.loc[:,"estimated_str"].min(), zmax=temp_population.loc[:,"estimated_str"].max()))
fig.update_layout(mapbox_style="basic", 
                  mapbox_accesstoken=os.getenv("MAPBOX_TOKEN"),
                  mapbox_zoom=5, 
                  mapbox_center={"lat": 4.389059008652357, "lon": 108.65244272591418})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
fig = go.Figure(go.Choroplethmapbox(geojson=geojson_data,
                                            locations=temp_population.loc[:,"parlimen"], 
                                            z=temp_population.loc[:,"str_count"],
                                            featureidkey=f"properties.parlimen",
                                            autocolorscale = False, 
                                            colorscale="rainbow",
                                            colorbar_title="STR Population",
                                            showlegend=False,
                                            text=None,
                                            zmin=temp_population.loc[:,"str_count"].min(), zmax=temp_population.loc[:,"str_count"].max(), 
                                            marker_opacity=0.5, 
                                            marker_line_width=1))
fig.update_layout(mapbox_style="basic", 
                    mapbox_accesstoken=os.getenv("MAPBOX_TOKEN"),
                    mapbox_zoom=5, 
                    mapbox_center={"lat": 4.389059008652357, "lon": 108.65244272591418},
                    )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
from math import radians, sin, cos, asin, sqrt
from function.file import file
from function.provider import gp
from function.population import population
from function.map import map
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

from dotenv import load_dotenv
import os

# To set the environement
load_dotenv()
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

df = pd.read_parquet("/Users/wh0102/Downloads/github/mph/geo_project/streamlit/data/population/Population District.parquet")\
.pivot_table(index="district", columns="date", values="population", aggfunc=sum, margins=True)
df.columns

In [26]:
from math import radians, sin, cos, asin, sqrt
from scipy.stats import skew, kurtosis, shapiro, norm, spearmanr, iqr
from function.file import file
from function.provider import gp
from function.population import population
from function.map import map
import geopandas as gpd
import pandas as pd
import polars as pl
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import numpy as np
from dotenv import load_dotenv
import os
# import pygmt
# class gp:
#     _district_code_list = ['14_1', '13_1', '12_7', '10_8', '10_1', '10_5', '10_2', '8_3', '7_4', '1_2']
#     _state_code_list = [14, 13, 12, 10, 8, 7, 1]
# To set the environement
load_dotenv("/Users/wh0102/Downloads/github/mph/geo_project/.env")
px.set_mapbox_access_token(os.getenv("MAPBOX_TOKEN"))

# geojson_data = map.read_geojson_file(file._map_district)

population = pd.read_parquet(file._population_str_ascii_gp_households).rename(columns={"X":'lon', "Y":"lat"})
target_population = population.query(f"code_state_district.isin({gp._district_code_list})")\
                              .reset_index().drop(columns="index").reset_index()
gp_df = pd.read_excel(file._gp_file)

target_population = target_population.query("distance > 5")
print(len(target_population))

population_fig = ff.create_hexbin_mapbox(
            data_frame=target_population.loc[:,("lat", "lon", "estimated_str")], 
            lat="lat", 
            lon="lon",
            color = "estimated_str",
            color_continuous_scale = "blues",
            agg_func=np.sum,
            nx_hexagon=1000, 
            opacity=0.3, 
            min_count=2, 
            show_original_data=False,
        )

# It wont show
# population_fig = px.scatter_mapbox(target_population, lat="lat", lon="lon", size="estimated_str", color="district")

gp_fig = go.Figure(go.Scattermapbox(
    mode = "markers+text",
    lon = gp_df["Longitude"], lat = gp_df["Latitude"],
    marker = {'size':10, 'symbol': "marker"},
    text = gp_df["clinic_name"],
    ))

gp_fig.add_trace(population_fig.data[0])

gp_fig.update_layout(
    mapbox = {
        'accesstoken': os.getenv("MAPBOX_TOKEN"),
        'style': "basic", 'zoom': 5, "center":{"lat": 4.389059008652357, "lon": 108.65244272591418}},
    showlegend = False, height=1200)

gp_fig.show()

3462


In [16]:
px.scatter_mapbox(target_population, lat="lat", lon="lon", size="estimated_str", color="district")

/Users/wh0102/Downloads/github/.venv/lib/python3.12/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
from shapely.geometry import Polygon
def grid_data(df, cell_size):
    """
    Grids population data into a GeoDataFrame.

    Args:
        df: Pandas DataFrame with columns 'x', 'y', and 'z'.
        cell_size: Size of grid cells in decimal degrees.

    Returns:
        GeoDataFrame with grid cells and population density.
    """

    # Calculate grid boundaries
    min_x, max_x = df['X'].min(), df['X'].max()
    min_y, max_y = df['Y'].min(), df['Y'].max()

    # Create grid points
    x_range = np.arange(min_x, max_x + cell_size, cell_size)
    y_range = np.arange(min_y, max_y + cell_size, cell_size)
    grid_points = [(x, y) for x in x_range for y in y_range]

    # Create grid polygons
    grid_polygons = []
    for i in range(len(x_range) - 1):
        for j in range(len(y_range) - 1):
            x1, y1 = grid_points[i * len(y_range) + j]
            x2, y2 = grid_points[(i + 1) * len(y_range) + j + 1]
            grid_polygons.append(Polygon([(x1, y1), (x2, y1), (x2, y2), (x1, y2)]))

    # Create GeoDataFrame
    gdf = gpd.GeoDataFrame(geometry=grid_polygons)

    return gdf

df = pd.read_csv("/Users/wh0102/Downloads/github/mph/geo_project/streamlit/data/population/mys_pd_2020_1km_ASCII_XYZ.csv")
grid_data(df, 1)

In [ ]:
import pygmt
import pandas as pd

df = pd.read_csv("/Users/wh0102/Downloads/github/mph/geo_project/streamlit/data/population/mys_pd_2020_1km_ASCII_XYZ.csv")

pygmt.xyz2grd(df, spacing="1k", region=[df["X"].min(), df["X"].max(), df["Y"].min(), df["Y"].max()])
